In [1]:
!pip install deepchem
import deepchem as dc
from deepchem.models.torch_models import MPNNModel

!pip install dgl-cu110
import dgl




     |████████████████████████████████| 608 kB 4.1 MB/s 
     |████████████████████████████████| 22.5 MB 1.0 MB/s 
     |████████████████████████████████| 39.9 MB 1.3 MB/s 
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [2]:
!pip install dgllife
import dgllife

     |████████████████████████████████| 138 kB 4.3 MB/s 
  Created wheel for dgllife: filename=dgllife-0.2.9-py3-none-any.whl size=219056 sha256=8fffa72b2c2524781a1adeae2dd919dc4b14d3c363ef2385a67e6d9b6d9b97f5
  Stored in directory: /root/.cache/pip/wheels/34/83/11/9772c74b559d9182c9083362e8ba8b0201c4963e41e03859fe
Successfully built dgllife


In [3]:
from deepchem.models.torch_models import MPNNModel

In [4]:
import pandas as pd
import sys
import os
import requests
import subprocess
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
smile_path = 'drive/MyDrive/Graph_Data_2/raw'
smile_name = 'tox21_PREPROCESSED.csv'

smile = pd.read_csv(os.path.join(smile_path, smile_name))
smile.head()

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,smiles
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CCN1C(=O)NC(c2ccccc2)C1=O
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC(O)(P(=O)(O)O)P(=O)(O)O


In [7]:
smile.shape

(7812, 13)

In [27]:
data_smiles = smile["NR-AR"].tolist()

In [28]:
#labels = ((smile.iloc[:, :-1]).to_numpy()).astype(float)
labels = smile["NR-AR"].tolist()
data_smiles = smile["smiles"].tolist()

print(labels[0])
print(data_smiles[0])

0.0
CCOc1ccc2nc(S(N)(=O)=O)sc2c1


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_smiles, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [30]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
X = featurizer.featurize(X_train)

dataset = dc.data.NumpyDataset(X=X, y=y_train)

# X_test_f = featurizer.featurize(X_val)
# test_dataset = dc.data.NumpyDataset(X=X_test_f, y=y_val)

X_test_f = featurizer.featurize(X_test)
test_dataset = dc.data.NumpyDataset(X=X_test_f, y=y_test)

In [31]:
metric1 = dc.metrics.Metric(dc.metrics.f1_score, np.mean)
metric2 = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
metric3 = dc.metrics.Metric(dc.metrics.accuracy_score, np.mean)

In [32]:
import numpy as np
np.random.seed(123)

model = MPNNModel(mode='classification', n_tasks=1,batch_size=64, learning_rate=.0001,edge_hidden_feats=128,num_step_message_passing=3)

loss =  model.fit(dataset, nb_epoch=200)

In [33]:
test_scores1 = model.evaluate(test_dataset, [metric1])
test_scores2 = model.evaluate(test_dataset, [metric2])
test_scores3 = model.evaluate(test_dataset, [metric3])

print("Test scores_f1")
print(test_scores1)

print("Test scores_auc")
print(test_scores2)

print("Test scores_acc")
print(test_scores3)

Test scores_f1
{'mean-f1_score': 0.43809523809523804}
Test scores_auc
{'mean-roc_auc_score': 0.7147751322751323}
Test scores_acc
{'mean-accuracy_score': 0.9622520793346129}


In [ ]:
#prediction on all classes together

In [13]:
from deepchem.molnet import load_tox21

In [18]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

In [14]:
tox21_tasks, tox21_datasets, transformers = load_tox21(featurizer=featurizer)
train_dataset, valid_dataset, test_dataset = tox21_datasets

Failed to featurize datapoint 95, [I-].[K+]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 255, [Hg+2]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 659, [Ba+2]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 985, [TlH2+]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 1423, [Cr+3]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 1534, [Fe+2]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to fe

In [19]:
import numpy as np
np.random.seed(123)

model = MPNNModel(mode='classification', n_tasks=12,batch_size=64, learning_rate=.0001,edge_hidden_feats=128,num_step_message_passing=3)

loss =  model.fit(train_dataset, nb_epoch=200)

In [20]:
test = model.predict(test_dataset)

print(type(test))

<class 'numpy.ndarray'>


In [21]:
print(y_test[64])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]


In [22]:
print(test[64])

[[0.65361565 0.34638438]
 [0.9281638  0.07183618]
 [0.03131154 0.9686885 ]
 [0.21399465 0.7860053 ]
 [0.36164686 0.6383531 ]
 [0.74916774 0.2508323 ]
 [0.63043916 0.3695608 ]
 [0.05215272 0.9478473 ]
 [0.17259611 0.8274039 ]
 [0.09654016 0.9034598 ]
 [0.05378142 0.94621855]
 [0.04683762 0.95316243]]


In [23]:
final = []
for i in range(test.shape[0]):
  res = []
  for j in range(test[i].shape[0]):
    if test[i][j][0] >= test[i][j][1]:
      res.append(0)
    else:
      res.append(1)
  arr = np.array(res) 
  final.append(arr)


test_true = np.vstack(final)

print(test_true.shape)
print(y_test.shape)

(1563, 12)
(1563, 12)


In [24]:
from sklearn.metrics import accuracy_score

accuracy_score(test_true, y_test)

0.508637236084453

In [25]:
from sklearn.metrics import roc_auc_score

roc_auc_score(test_true, y_test)

0.6172194498651851